In [1]:
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, ensemble, tree
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score,GridSearchCV,cross_validate
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100, 'display.max_columns', 400)


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import joblib
from mlxtend.plotting import plot_confusion_matrix

import warnings
warnings.filterwarnings("ignore")


import theano
import theano.tensor as Tensor
import random
import sys
import csv
import time
import utils
import visualize
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import regularizers
import re
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Dense, Flatten 

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
#Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed,RNN # for creating layers inside the Neural Network
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Tensorflow/Keras: 2.9.0


In [3]:
data4 = pd.read_csv('data4.csv')

In [4]:
data4 = data4.drop(['Unnamed: 0'], axis = 1)

In [5]:
train,test= train_test_split(data4, test_size=0.3,random_state=123, stratify=data4.Result)# stratify the outcome
X_train =train.drop(['Result'], axis = 1)
X_test = test.drop(['Result'], axis = 1)
Y_train = train['Result'].values
Y_test = test['Result'].values

In [6]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0)))
  
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, Y_train_res = sm.fit_resample(X_train, Y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(Y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(Y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(Y_train_res == 0)))

Before OverSampling, counts of label '1': 202311
Before OverSampling, counts of label '0': 721368 

After OverSampling, the shape of train_X: (1442736, 63)
After OverSampling, the shape of train_y: (1442736,) 

After OverSampling, counts of label '1': 721368
After OverSampling, counts of label '0': 721368


In [7]:
X_train_res= np.expand_dims(X_train_res, axis=2)

In [8]:
X_train_res.shape

(1442736, 63, 1)

In [9]:
X_test = np.expand_dims(X_test, axis=2)

In [10]:
X_test.shape

(395863, 63, 1)

In [12]:
import keras.backend as K
from keras import models 

def f1_score(y_true, y_pred):

    #Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    #If there are no true samples, fix the F1 score at 0.
    if c3 == 0.0:
        return 0.0

    #How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score 

In [20]:
model = keras.Sequential()
# RNN LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
model.add(layers.SimpleRNN(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
model.add(BatchNormalization())
model.add(layers.SimpleRNN(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(layers.SimpleRNN(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
# ---------------------------------------------------------------------------------------------------
# OPTIMIZER SETTINGS
opt = tf.keras.optimizers.Adam(learning_rate=0.0017, decay=1e-6)

# MODEL COMPILE
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','Recall','Precision',f1_score])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 63, 128)           16640     
                                                                 
 dropout_6 (Dropout)         (None, 63, 128)           0         
                                                                 
 batch_normalization_6 (Batc  (None, 63, 128)          512       
 hNormalization)                                                 
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 63, 128)           32896     
                                                                 
 dropout_7 (Dropout)         (None, 63, 128)           0         
                                                                 
 batch_normalization_7 (Batc  (None, 63, 128)          512       
 hNormalization)                                      

In [16]:
# RUN THE MODEL
history = model.fit(X_train_res, Y_train_res, epochs=5, batch_size=512, verbose=1,
                    validation_data=(X_test, Y_test))

Epoch 1/5
2818/2818 [==============================] - 625s 221ms/step - loss: 0.5567 - accuracy: 0.6952 - recall: 0.6770 - precision: 0.7025 - f1_score: 0.6894 - val_loss: 0.5399 - val_accuracy: 0.7246 - val_recall: 0.6112 - val_precision: 0.4130 - val_f1_score: 0.4918
Epoch 2/5
2818/2818 [==============================] - 627s 222ms/step - loss: 0.4710 - accuracy: 0.7549 - recall: 0.7603 - precision: 0.7521 - f1_score: 0.7563 - val_loss: 0.4094 - val_accuracy: 0.7880 - val_recall: 0.8055 - val_precision: 0.5101 - val_f1_score: 0.6239
Epoch 3/5
2818/2818 [==============================] - 624s 221ms/step - loss: 0.3727 - accuracy: 0.8166 - recall: 0.8210 - precision: 0.8138 - f1_score: 0.8173 - val_loss: 0.3071 - val_accuracy: 0.8520 - val_recall: 0.8044 - val_precision: 0.6263 - val_f1_score: 0.7036
Epoch 4/5
2818/2818 [==============================] - 629s 223ms/step - loss: 0.3273 - accuracy: 0.8432 - recall: 0.8529 - precision: 0.8368 - f1_score: 0.8446 - val_loss: 0.3052 - val_a

In [19]:
# RUN THE MODEL
history = model.fit(X_train_res, Y_train_res, epochs=10, batch_size=512, verbose=1,
                    validation_data=(X_test, Y_test))

Epoch 1/10
2818/2818 [==============================] - 631s 223ms/step - loss: 0.5613 - accuracy: 0.6907 - recall: 0.6718 - precision: 0.6981 - f1_score: 0.6846 - val_loss: 0.4364 - val_accuracy: 0.8011 - val_recall: 0.3415 - val_precision: 0.5776 - val_f1_score: 0.4276
Epoch 2/10
2818/2818 [==============================] - 620s 220ms/step - loss: 0.4752 - accuracy: 0.7537 - recall: 0.7599 - precision: 0.7506 - f1_score: 0.7552 - val_loss: 0.4557 - val_accuracy: 0.7431 - val_recall: 0.8723 - val_precision: 0.4549 - val_f1_score: 0.5973
Epoch 3/10
2818/2818 [==============================] - 616s 218ms/step - loss: 0.3885 - accuracy: 0.8063 - recall: 0.8171 - precision: 0.7999 - f1_score: 0.8082 - val_loss: 0.3341 - val_accuracy: 0.8361 - val_recall: 0.8418 - val_precision: 0.5878 - val_f1_score: 0.6914
Epoch 4/10
2818/2818 [==============================] - 1221s 433ms/step - loss: 0.3801 - accuracy: 0.8090 - recall: 0.8182 - precision: 0.8034 - f1_score: 0.8107 - val_loss: 0.4544 - 

In [21]:
# RUN THE MODEL
history = model.fit(X_train_res, Y_train_res, epochs=5, batch_size=512, verbose=1,
                    validation_data=(X_test, Y_test))

Epoch 1/5
2818/2818 [==============================] - 644s 227ms/step - loss: 0.5868 - accuracy: 0.6695 - recall: 0.6456 - precision: 0.6781 - f1_score: 0.6611 - val_loss: 0.6981 - val_accuracy: 0.4996 - val_recall: 0.8679 - val_precision: 0.2873 - val_f1_score: 0.4311
Epoch 2/5
2818/2818 [==============================] - 631s 224ms/step - loss: 0.5826 - accuracy: 0.6749 - recall: 0.6718 - precision: 0.6760 - f1_score: 0.6736 - val_loss: 0.5265 - val_accuracy: 0.7204 - val_recall: 0.6218 - val_precision: 0.4091 - val_f1_score: 0.4926
Epoch 3/5
2818/2818 [==============================] - 641s 228ms/step - loss: 0.6065 - accuracy: 0.6461 - recall: 0.6427 - precision: 0.6471 - f1_score: 0.6445 - val_loss: 0.5298 - val_accuracy: 0.7910 - val_recall: 0.3221 - val_precision: 0.5382 - val_f1_score: 0.4014
Epoch 4/5
2818/2818 [==============================] - 637s 226ms/step - loss: 0.6060 - accuracy: 0.6477 - recall: 0.6351 - precision: 0.6516 - f1_score: 0.6425 - val_loss: 0.5661 - val_a

In [22]:
model = keras.Sequential()
# RNN LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
model.add(layers.SimpleRNN(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
model.add(BatchNormalization())
model.add(layers.SimpleRNN(128, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(layers.SimpleRNN(64, input_shape=(X_train_res.shape[1],X_train_res.shape[2]), activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
# ---------------------------------------------------------------------------------------------------
# OPTIMIZER SETTINGS
opt = tf.keras.optimizers.Adam(learning_rate=0.0017, decay=1e-6)

# MODEL COMPILE
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','Recall','Precision'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_9 (SimpleRNN)    (None, 63, 128)           16640     
                                                                 
 dropout_9 (Dropout)         (None, 63, 128)           0         
                                                                 
 batch_normalization_9 (Batc  (None, 63, 128)          512       
 hNormalization)                                                 
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 63, 128)           32896     
                                                                 
 dropout_10 (Dropout)        (None, 63, 128)           0         
                                                                 
 batch_normalization_10 (Bat  (None, 63, 128)          512       
 chNormalization)                                     